# Purple Team Playbook Master
## Use Case: Master Playbook for Purple Team engagement

### LogRhythm Labs
### July 2020

##### Functionality:
##### Create Master LR Case for Purple Team Exercise with custom Playbook for list of techniques
##### Create Per-Technique LR Cases with a technique-level playbook

### Please configure the server connection parameters under "Connection settings for the LR Case API"



#### Import required libraries

In [ ]:
from attackcti import attack_client
import requests
from pandas import *
import json
import urllib3 # to suppress SSL warnings if a self-signed cert is being used to access the Case API
urllib3.disable_warnings() # to suppress SSL warnings if a self-signed cert is being used to access the Case API
import pyinputplus as pyip


#### Connection settings for the LR Case API

In [ ]:
# In order to use the LogRhythm Case API you will need to setup an API account and generate a Case API token. Please refer to https://docs.logrhythm.com/docs/lrapi/rest-api/case-api/set-up-the-case-api for more information.

server = 'server' # replace 'server' with the name or IP address of the machine running Case API (usually the Web Console server)

TagsEndpoint = 'https://{}:8501/lr-case-api/tags'.format(server)
RequestHeaders = {} #set up the dictionary for request headers to the LogRhythm Case API
RequestHeaders['Authorization'] = 'Bearer CaseAPItoken' # replace 'caseAPItoken' with the token generated for the Case API user
RequestHeaders['Content-Type'] = 'application/json'


#### Load MITRE ATT&CK Enterprise Matrix

In [ ]:
lift = attack_client()
%time all_enterprise = lift.get_enterprise()


#### Create Case tags from the ID and name of all tactics in the Enterprise Matrix 

In [ ]:
## create Case Tags for each MITRE tactic. Also create a lookup table (a dict) so that when a case is created for a technique the tactic tag can be looked up and added.
tacticTagLookup = {}
for tactic in all_enterprise['tactics']:
    tacticTag = '{"text":"'+tactic['external_references'][0]['external_id']+': '+tactic['name'] + '"}'
    tacticTagResponse = requests.post(TagsEndpoint, verify = False, headers=RequestHeaders,data = tacticTag)
    tacticTagLookup[tactic['name'].lower()] = tacticTagResponse.json()['number']
    print(tacticTag)
    

#### Red Team: Input list of MITRE techniques that will be simulated in purple team exercise

In [ ]:
from ipywidgets import widgets
from IPython.display import display
# create list of techniques
techniqueIDs=[]
numOfTechniques = int(input("Enter number of techniques to simulate: "))
# fill in list
for i in range(0, numOfTechniques):
    ele = str(input("Enter technique number : "))
    techniqueIDs.append(ele) # adding the element
# print list to check (if needed uncomment next line)
# print(techniqueIDs)

#### Prompt the user for a name for the Purple team exercise and make a case tag for it

In [ ]:
purpleExerciseName = input("Please enter a name for the Purple Team exercise")

#### Create a purple team case tag for the exercise

In [ ]:
purpleExerciseTagName = "PurpleTeam: " + purpleExerciseName
purpleExerciseTagData = '{"text":"'+purpleExerciseTagName +'"}'
purpleExerciseTagResponse = requests.post(TagsEndpoint, verify = False, headers=RequestHeaders,data = purpleExerciseTagData)
purpleExerciseTagNumber = purpleExerciseTagResponse.json()['number']



#### Create general PurpleTeam tag

In [ ]:
purpleTagName = "PurpleTeam"
purpleTagData = '{"text":"'+purpleTagName +'"}'
purpleTagResponse = requests.post(TagsEndpoint, verify = False, headers=RequestHeaders,data = purpleTagData)
purpleTagNumber = purpleTagResponse.json()['number']



In [ ]:
print(purpleTagNumber)
print(purpleExerciseTagNumber)

#### Create a master playbook 

In [ ]:
playbooksEndpoint = 'https://{}:8501/lr-case-api/playbooks/'.format(server) 
playbookName = "Master Playbook: " + purpleExerciseName
playbookData = '{"name":"' + playbookName + '","description": "Master playbook for purple team exercise","permissions": {"read": "publicAllUsers","write": "publicAllUsers"},"entities": []}'
playbookResponse = requests.post(playbooksEndpoint, verify = False, headers=RequestHeaders,data = playbookData)


In [ ]:
print(playbookResponse.json())

In [ ]:
playbookID = playbookResponse.json()['id']

In [ ]:
print(playbookID)

#### Create the procedures for the master playbook 

In [ ]:

masterProcedures = []

for t in techniqueIDs:
    print("building procedures for technique",t)
    masterProcedures.append({"name": "Red Team: Simulate technique {}".format(t),"description": "Red Team runs simulation of MITRE ATT&CK technique {}".format(t)})
    masterProcedures.append({"name": "Blue Team: Identify evidence of technique {} execution".format(t),"description": "Identify whether existing analytics detect execution of MITRE technique {}. If not, perform investigations to find evidence of technique execution.".format(t)})
    masterProcedures.append({"name": "Blue Team: Generate a LogRhythm Case for technique {} with Jupyter".format(t),"description": "Using the Purple Team Jupyter Notebook generate a LogRhythm Case for technique {}".format(t)})
    masterProcedures.append({"name": "Blue Team: Navigate to the LogRhythm Case for technique {} and follow the technique playbook".format(t),"description": "Navigate to the Case generated for technique {} and follow the procedures in the playbook attached to the Case".format(t)})
    masterProcedures.append({"name": "Blue team: If required, document and automate new detection methods for technique {}.".format(t),"description": "Any new detection techniques, regardless of efficacy, should be documented. Effective detection techniques should be automated through the creation of AIE Engine rules, saved investigations, dashboards or playbooks depending on false positive and false negative probability."})

masterProcedures.append({"name": "Set the Master Case status to complete","description": "Once all procedures in this playbook are complete, set the status of this Case to completed"})

proceduresData = json.dumps(masterProcedures)



In [ ]:
print(proceduresData)

#### Update the Purple Team Master playbook with procedures for each technique

In [ ]:
procedureUpdateEndpoint = 'https://{}:8501/lr-case-api/playbooks/{}/procedures/'.format(server,playbookID)
procedureUpdateResponse = requests.put(procedureUpdateEndpoint, verify = False, headers=RequestHeaders,data = proceduresData)


#### Create a master purple team exercise case with the tag of the exercise name and the playbook walking through the techniques

In [ ]:
CaseEndpoint = 'https://{}:8501/lr-case-api/cases'.format(server)
masterCaseName = "Purple Team: "+ purpleExerciseName + ": Master Case"
createCaseData = '{"name":"'+masterCaseName+'","priority": 5}'
createCaseDataResponse = requests.post(CaseEndpoint, verify = False, headers = RequestHeaders,data = createCaseData)
masterCaseNumber = createCaseDataResponse.json()['id']
       


In [ ]:
print(masterCaseNumber)

#### Attach the master playbook to the master case

In [ ]:
masterPlaybookEndpoint = 'https://{}:8501/lr-case-api/cases/'.format(server)+ masterCaseNumber + '/playbooks/'
addPlaybookData = '{"id":"'+playbookID+'"}'
addPlaybookResponse = requests.post(masterPlaybookEndpoint, verify = False, headers = RequestHeaders, data = addPlaybookData)
print(addPlaybookResponse.json())


#### Add the custom purple team tag to the Master Case

In [ ]:
masterTagEndpoint = 'https://{}:8501/lr-case-api/cases/'.format(server)+ masterCaseNumber + '/actions/addTags/'
masterTagData = '{"numbers": [' + str(purpleTagNumber)+','+str(purpleExerciseTagNumber)+']}'
addMasterTagResponse = requests.put(masterTagEndpoint,verify = False, headers = RequestHeaders, data = masterTagData)
print(addMasterTagResponse.json())

In [ ]:
print(masterTagEndpoint)

#### Create playbook for technique cases

In [ ]:
playbooksEndpoint = 'https://{}:8501/lr-case-api/playbooks/'.format(server)
playbookName = "Purple Exercise MITRE technique playbook" 
playbookData = '{"name":"' + playbookName + '","description": "Per-MITRE-technique playbook for purple team exercise","permissions": {"read": "publicAllUsers","write": "publicAllUsers"},"entities": []}'
playbookResponse = requests.post(playbooksEndpoint, verify = False, headers=RequestHeaders,data = playbookData)
print(playbookResponse.json())
techniquePlaybookID = playbookResponse.json()['id']

#### Create procedures to the technique case

In [ ]:
techniqueProcedures = []

techniqueProcedures.append({"name": "Blue team: Add evidence to Case","description": "Add logs and alarms relevant to the MITRE technique resulting from an analytic drill down or threat hunting"})
techniqueProcedures.append({"name": "Blue team: Set Case status to incident","description": "Set Case status to incident"})
techniqueProcedures.append({"name": "Blue team: Decide if mitigating steps must be taken","description": "If necessary, follow playbooks for threat mitigation. Ensure that the Case status is set to mitigated if appropriate."})
techniqueProcedures.append({"name": "Blue team: Set Case status to resolved by clicking mitigated and then resolved when evidence collection is complete","description": "When mitigation steps are complete, set the Case status to resolved so that Case statistics are calculated properly"})

techniqueProceduresData = json.dumps(techniqueProcedures)

#### Add technique procedures to technique playbook

In [ ]:
procedureUpdateEndpoint = 'https://{}:8501/lr-case-api/playbooks/{}/procedures/'.format(server,techniquePlaybookID)
procedureUpdateResponse = requests.put(procedureUpdateEndpoint, verify = False, headers=RequestHeaders,data = techniqueProceduresData)


 
### Create Case for each technique that will be executed in the Purple Team exercise
#### Tag Case with name of exercise, technique, tactic and data source requirements

#### Present User with choice of which technique will have a case created

In [ ]:
technique = pyip.inputMenu(techniqueIDs,numbered=True)
print("Building a Case for technique {}".format(technique))

In [ ]:
CaseEndpoint = 'https://{}:8501/lr-case-api/cases'.format(server)


#iterate through the techniques and come up with list of technique tags
for t in all_enterprise['techniques']:
    if t['external_references'][0]['external_id'] == technique:
        #
        #CREATE A NAME FOR THE CASE. Example "T1015: Accessibility Features" 
        #
        createCaseData = '{"name": "'+t['external_references'][0]['external_id']+': '+t['name']+'","priority": 4}'
        #
        #CREATE THE CASE AND CAPTURE THE CASE NUMBER.
        #
        createCaseDataResponse = requests.post(CaseEndpoint, verify = False, headers = RequestHeaders,data = createCaseData)
        caseNumber = createCaseDataResponse.json()['number']
        caseID = createCaseDataResponse.json()['id']
        #
        #CREATE A NAME FOR THE TECHNIQUE TAG. Example "T1496: Resource Hijacking"
        #
        techniqueTag = '{"text":"'+t['external_references'][0]['external_id']+": "+t['name']+'"}'
        #
        #CREATE THE TECHNIQUE TAG AND CAPTURE THE TAG NUMBER
        #
        techniqueTagResponse = requests.post(TagsEndpoint, verify = False, headers=RequestHeaders,data = techniqueTag)
        techniqueTagNumber = techniqueTagResponse.json()['number']
        print(techniqueTag,techniqueTagNumber,caseNumber)
        #
        #BUILD A URL FOR ADDING TAGS TO A CASE 
        #
        caseTagEndpoint = CaseEndpoint + '/' + str(caseNumber) + '/actions/addTags'
        #
        #adding tags is accomplished by sending a list of the tag numbers to be sent
        #
        # QUERY THE TACTIC NAME FROM THE TECHNIQUE. QUERY THE TAG NUMBER FOR THE TACTIC
        #
        tacticName = t['kill_chain_phases'][0]['phase_name'].replace('-',' ').lower()
        caseTagData = '{"numbers":['+ str(techniqueTagNumber)+','+str(tacticTagLookup[tacticName])+']}'
        requests.put(caseTagEndpoint, verify = False, headers=RequestHeaders, data=caseTagData) 
        #
        # ADD TAGS FOR EACH OF THE DATA SOURCES REQUIRED FOR DETECTION OF THE TECHNIQUE
        #
        for dataSource in t['x_mitre_data_sources']:
            dataSourceTag = '{"text": "Data Source: ' + dataSource + '"}'
            dataSourceTagResponse = requests.post(TagsEndpoint, verify = False, headers=RequestHeaders,data = dataSourceTag)
            dataSourceTagNumber = dataSourceTagResponse.json()['number']
            dataSourceTagData = '{"numbers":['+str(dataSourceTagNumber)+']}'
            requests.put(caseTagEndpoint, verify = False, headers=RequestHeaders, data = dataSourceTagData)
        #
        #
        # ADD A TAG FOR THE PURPLE TEAM EXERCISE NAME
        #
        addPurpleTagData = '{"numbers":['+ str(purpleTagNumber)+','+str(purpleExerciseTagNumber)+']}'
        purpleTagResponse = requests.put(caseTagEndpoint, verify = False, headers=RequestHeaders,data = addPurpleTagData)
        #
        # BUILD A URL FOR ADDING NOTES TO THE CASE
        # create endpoint for adding note to the case 
        #
        caseNoteEndpoint = CaseEndpoint +'/' + str(caseNumber)+'/evidence/note/'
        #
        #BUILD A URL TO CHECK IF AN ATOMIC RED TEAM TEST EXISTS FOR THE TECHNIQUE
        #
        ARTEndpoint = 'https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/'+ t['external_references'][0]['external_id']
        #
        # IF AN ATOMIC RED TEAM TEST EXISTS ADD THE URL AS A NOTE TO THE CASE
        #
        ARTSiteCheck = requests.get(ARTEndpoint)
        if ARTSiteCheck.status_code == 200:
            caseNoteData = '{"text":"Atomic Red Team test for this technique can be found at: '+ARTEndpoint+'"}'
            noteResponse = requests.post(caseNoteEndpoint, verify = False, headers=RequestHeaders, data=caseNoteData) 
        #
        # Associate the technique case with the Master Case
        #
        caseAssocEndpoint = CaseEndpoint + '/' + caseID + '/associated'
        caseAssocData = '{"ids": ["'+masterCaseNumber+'"]}'
        caseAssocResponse = requests.post(caseAssocEndpoint,verify = False, headers=RequestHeaders,data=caseAssocData)
        #
        #
        # Add the per-technique playbook to the case
        #
        techniquePlaybookEndpoint = 'https://{}:8501/lr-case-api/cases/'.format(server)+ caseID + '/playbooks/'
        addTechniquePlaybookData = '{"id":"'+techniquePlaybookID+'"}'
        addTechniquePlaybookResponse = requests.post(techniquePlaybookEndpoint, verify = False, headers = RequestHeaders, data = addTechniquePlaybookData)
        print(addTechniquePlaybookResponse.json())

            
